# Demo — mcport (src layout)

In [ ]:

import sys
from pathlib import Path
BASE = Path.cwd()
src_path = BASE / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))


In [ ]:

import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
sns.set_theme(context="notebook", style="whitegrid")
from mcport import PriceSeries, Portfolio, MonteCarloSimulation, MonteCarloPlots, MonteCarloReport
print("Imports OK")


In [ ]:

idx = pd.bdate_range('2022-01-03','2024-12-31')
def gbm(n, mu=0.1, sigma=0.2, s0=100, seed=0):
    rng=np.random.default_rng(seed); dt=1/252
    z=rng.standard_normal(n-1)
    path=np.r_[0, np.cumsum((mu-0.5*sigma**2)*dt + sigma*np.sqrt(dt)*z)]
    return s0*np.exp(path)
a = PriceSeries(symbol="ASSET_A", asset_type="equity", currency="USD", provider="sim",
                data=pd.DataFrame({"price":gbm(len(idx),0.12,0.25,100,1)}, index=idx))
b = PriceSeries(symbol="ASSET_B", asset_type="equity", currency="USD", provider="sim",
                data=pd.DataFrame({"price":gbm(len(idx),0.06,0.12, 50,2)}, index=idx))
portfolio = Portfolio(positions=[a,b], weights=[0.6,0.4], name="Demo 60_40", currency="USD")


In [ ]:

mc = MonteCarloSimulation(price_series=portfolio)
summ = mc.simulate_and_summarize(days=252, n_sims=500, seed=7)
plots = MonteCarloPlots(mc)
plots.plot_history()
plots.plot_history_with_simulations(summ["prices"])
plots.plot_final_hist(summ["prices"])
plt.show()
